Embeddings Huggging face, smilarity search, cosine and Nuclean distance similarity, FAISS vector, index(flat, HNSW, IVF), Retriever

In [9]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [11]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
embedding.embed_query("This is a test query")

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
documents=["what is the capital of France?",
           "what is the capital of USA?",
           "who is the Prime minister of India?",
           "who is the president of USA?"]

In [17]:
mq_query = "Narendra Modi is the Prime Minister of India."

In [18]:
document_embedding = embedding.embed_documents(documents)

In [21]:
query_embedding = embedding.embed_query(mq_query)

In [22]:
#it checks nearest similar query
cosine_similarity([query_embedding], document_embedding)

array([[0.11377665, 0.07118423, 0.76168932, 0.33584692]])

In [23]:
from sklearn.metrics.pairwise import euclidean_distances

In [24]:
#it checks nearest distance
euclidean_distances([query_embedding], document_embedding)

array([[1.3313327 , 1.3629496 , 0.69037769, 1.15252163]])

In [25]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [26]:
index = faiss.IndexFlatL2(384)

In [28]:
vector_store= FAISS(
    embedding_function= embedding,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [29]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

In [30]:
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10
]

In [31]:
index = faiss.IndexFlatL2(384)
vector_store = FAISS(
    embedding_function=embedding,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
vector_store.add_documents(documents)

In [ ]:
vector_store.similarity_search(
    "What is the best framework for building stateful, agentic applications?",k=2)

In [ ]:
vector_store.similarity_search(
    query="What is the best framework for building stateful, agentic applications?", 
    filter={"source": "tweet"}
)

In [43]:
result = vector_store.similarity_search(
    query="What is the best framework for building stateful, agentic applications?", 
    filter={"source":"news"}
)

In [45]:
result[0].page_content

'The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'

In [49]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [50]:
retriever.invoke(
    "What is the best framework for building stateful, agentic applications?")

[Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [51]:
vector_store.save_local("faiss_index")

In [56]:
new_vector_store = FAISS.load_local(
    "faiss_index",
   embedding, allow_dangerous_deserialization=True)

In [57]:
new_vector_store.similarity_search(
    "What is the best framework for building stateful, agentic applications?")

[Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!')]

In [58]:
new_vector_store.as_retriever(search_kwargs={"k": 2}).invoke(
    "What is the best framework for building stateful, agentic applications?")

[Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [59]:
from langchain_community.document_loaders import PyPDFLoader

In [76]:
file_path = r"/Users/prayagsonar/Downloads/Projects/AgenticAI/2-Langchain_Basics/Class_3/vectorStore/FAISS/llama2.pdf"

In [77]:
loader = PyPDFLoader(file_path)

In [90]:
pages = loader.load()

In [91]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [93]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, #hyperparameter
    chunk_overlap=100,)

In [95]:
split_docs = splitter.split_documents(pages)

In [96]:
len(split_docs)

652

In [97]:
index = faiss.IndexFlatL2(384)
vector_store = FAISS(
    embedding_function=embedding,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
vector_store.add_documents(split_docs)

In [100]:
retriever= vector_store.as_retriever(search_kwargs={"k": 10})

In [ ]:
retriever.invoke(
    "what is the llama model")